In [1]:
import pandas as pd
import numpy  as np
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell as IS
IS.ast_node_interactivity = "all"

2023-10-27 18:19:37.766370: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-27 18:19:37.805446: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-27 18:19:37.805473: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-27 18:19:37.806443: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-27 18:19:37.812317: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-27 18:19:37.812709: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
import pandas as pd
churn = pd.read_csv('data/Churn_Modelling.csv')

In [3]:
df = churn.copy()
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
X = df.iloc[:, 3:13]
Y = df.iloc[:, 13]
X, Y

(      CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
 0             619    France  Female   42       2       0.00              1   
 1             608     Spain  Female   41       1   83807.86              1   
 2             502    France  Female   42       8  159660.80              3   
 3             699    France  Female   39       1       0.00              2   
 4             850     Spain  Female   43       2  125510.82              1   
 ...           ...       ...     ...  ...     ...        ...            ...   
 9995          771    France    Male   39       5       0.00              2   
 9996          516    France    Male   35      10   57369.61              1   
 9997          709    France  Female   36       7       0.00              1   
 9998          772   Germany    Male   42       3   75075.31              2   
 9999          792    France  Female   28       4  130142.79              1   
 
       HasCrCard  IsActiveMember  EstimatedSalary 

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X['Geography'] = le.fit_transform(X['Geography'])
X['Gender']    = le.fit_transform(X['Gender'])

In [6]:
X.isna().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64

In [7]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose       import ColumnTransformer
ohe = ColumnTransformer([("ohe", OneHotEncoder(dtype=float), [1])], remainder="passthrough")

X = ohe.fit_transform(X)
X = X[:, 1:]

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import StandardScaler

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=0)
sc = StandardScaler()
X_train = sc.fit_transform(x_train)
X_test  = sc.fit_transform(x_test)

In [10]:
X_train, X_test

(array([[-0.56953701, -0.57436296, -1.78451879, ..., -1.56299709,
         -1.03339307,  1.0890591 ],
        [-0.56953701, -0.57436296,  1.47401668, ...,  0.63979645,
          0.967686  , -0.4551902 ],
        [-0.56953701, -0.57436296, -0.82854466, ..., -1.56299709,
          0.967686  , -0.65446947],
        ...,
        [-0.56953701, -0.57436296,  0.89837634, ...,  0.63979645,
         -1.03339307,  1.40745917],
        [-0.56953701,  1.74105933, -0.62295883, ...,  0.63979645,
          0.967686  ,  0.83939459],
        [ 1.75581215, -0.57436296, -0.2837422 , ...,  0.63979645,
         -1.03339307,  0.31973777]]),
 array([[ 1.67387076, -0.57268474, -0.55982398, ...,  0.65890748,
          0.97545271,  1.62753708],
        [-0.59741769, -0.57268474, -1.33593594, ...,  0.65890748,
         -1.02516503,  0.51106949],
        [-0.59741769,  1.74616143,  0.58336795, ...,  0.65890748,
          0.97545271, -0.4108104 ],
        ...,
        [-0.59741769,  1.74616143,  0.35263197, ..., -

In [25]:
from xgboost import XGBClassifier
classifier = XGBClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_pred, y_test)
cm

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

array([[12,  0,  0],
       [ 0,  9,  1],
       [ 0,  1,  7]])